In [1]:
import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import matplotlib.pyplot as plt
from IPython.display import IFrame
alt.renderers.enable('mimetype')
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [2]:
game_stat = pd.read_csv('data/game_statistics.csv')

In [3]:
game_stat['Team'].unique()

array(['ANA', 'L.A', 'CAR', 'MTL', 'DET', 'OTT', 'TOR', 'COL', 'DAL',
       'FLA', 'NYR', 'N.J', 'T.B', 'NSH', 'CGY', 'PHI', 'EDM', 'S.J',
       'PHX', 'STL', 'PIT', 'CBJ', 'BUF', 'NYI', 'ATL', 'WSH', 'BOS',
       'VAN', 'MIN', 'CHI', 'WPG', 'ARI', 'VGK', 'SEA'], dtype=object)

In [4]:
game_stat.columns

Index(['Game_ID', 'Season', 'Team', 'Date', 'Is_home', 'Player_Id', 'Name',
       'Pos.', 'Age', 'Game_No.', 'TOI_All', 'TOI_EV', 'TOI_PP', 'TOI_SH',
       'EV_G', 'EV_A1', 'EV_A2', 'EV_ixG', 'EV_GF', 'EV_GA', 'EV_xGF',
       'EV_xGA', 'EV_SF', 'EV_SA', 'EV_FF', 'EV_FA', 'EV_CF', 'EV_CA', 'PP_G',
       'PP_A1', 'PP_A2', 'PP_ixG', 'PP_GF', 'PP_GA', 'PP_xGF', 'PP_xGA',
       'PP_SF', 'PP_SA', 'PP_FF', 'PP_FA', 'PP_CF', 'PP_CA', 'SH_G', 'SH_A1',
       'SH_A2', 'SH_ixG', 'SH_GF', 'SH_GA', 'SH_xGF', 'SH_xGA', 'SH_SF',
       'SH_SA', 'SH_FF', 'SH_FA', 'SH_CF', 'SH_CA', 'PEN_take', 'PEN_draw'],
      dtype='object')

TOI is in seconds

In [5]:
game_stat_2122 = game_stat[(game_stat['Season']==20212022)&(game_stat['Pos.']!='G')]

In [6]:
game_stat_2122['Date'] = pd.to_datetime(game_stat_2122['Date'],format = '%d/%m/%Y')
game_stat_2122['TOI_All'] = game_stat_2122['TOI_All']/60

/var/folders/cp/sy05qy250mg99nnwmfjffrwh0000gn/T/ipykernel_9729/3975719907.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_stat_2122['Date'] = pd.to_datetime(game_stat_2122['Date'],format = '%d/%m/%Y')
/var/folders/cp/sy05qy250mg99nnwmfjffrwh0000gn/T/ipykernel_9729/3975719907.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_stat_2122['TOI_All'] = game_stat_2122['TOI_All']/60


In [7]:
pen_game_stat = game_stat_2122.groupby('Name')['TOI_All','PEN_take','PEN_draw'].sum().reset_index()

/var/folders/cp/sy05qy250mg99nnwmfjffrwh0000gn/T/ipykernel_9729/949682644.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pen_game_stat = game_stat_2122.groupby('Name')['TOI_All','PEN_take','PEN_draw'].sum().reset_index()


In [8]:
pen_game_stat = pen_game_stat[pen_game_stat['TOI_All']>1000]

In [13]:
pen_game_stat['PEN_take_eff'] = pen_game_stat['PEN_take']*60/pen_game_stat['TOI_All']
pen_game_stat['PEN_draw_eff'] = pen_game_stat['PEN_draw']*60/pen_game_stat['TOI_All']
pen_game_stat['PEN_ratio'] = pen_game_stat['PEN_draw']/(pen_game_stat['PEN_draw']+pen_game_stat['PEN_take'])

In [14]:
pen_game_stat.sort_values('PEN_ratio',ascending = False).head(10)

,Name,TOI_All,PEN_take,PEN_draw,PEN_take_eff,PEN_draw_eff,PEN_ratio
596,Marcus Johansson,1164.650000,4,22,0.206070,1.133388,0.846154
843,Ryan Pulock,1140.750000,2,10,0.105194,0.525970,0.833333
296,Elias Pettersson,1419.950000,12,58,0.507060,2.450791,0.828571
361,Isac Lundestrom,1199.100000,8,34,0.400300,1.701276,0.809524
841,Ryan O'Reilly,1639.266667,14,58,0.512424,2.122900,0.805556
439,Jesper Fast,1358.950000,4,16,0.176607,0.706428,0.800000
218,Connor Brown,1230.983333,10,36,0.487415,1.754695,0.782609
205,Cole Caufield,1075.633333,10,34,0.557811,1.896557,0.772727
163,Calle Jarnkrok,1184.216667,6,20,0.303998,1.013328,0.769231
467,Johnny Gaudreau,1697.316667,18,58,0.636298,2.050295,0.763158


In [15]:
pen_game_stat.sort_values('PEN_take_eff',ascending = False).head(10)

,Name,TOI_All,PEN_take,PEN_draw,PEN_take_eff,PEN_draw_eff,PEN_ratio
894,Tanner Jeannot,1303.333333,120,100,5.524297,4.603581,0.454545
763,Patrick Maroon,1201.300000,106,92,5.294265,4.595022,0.464646
595,Marcus Foligno,1217.766667,89,78,4.385077,3.843101,0.467066
583,Luke Kunin,1143.383333,83,63,4.355495,3.305978,0.431507
311,Evander Kane,1086.966667,77,40,4.250360,2.207979,0.341880
679,Milan Lucic,1101.066667,75,51,4.086946,2.779123,0.404762
118,Brady Tkachuk,1399.566667,93,87,3.986948,3.729726,0.483333
791,Radko Gudas,1509.116667,99,75,3.936077,2.981877,0.431034
382,Jacob Middleton,1310.266667,84,75,3.846545,3.434415,0.471698
847,Sam Bennett,1323.783333,82,58,3.716620,2.628829,0.414286


In [16]:
pen_game_stat.sort_values('PEN_draw_eff',ascending = False).head(10)

,Name,TOI_All,PEN_take,PEN_draw,PEN_take_eff,PEN_draw_eff,PEN_ratio
894,Tanner Jeannot,1303.333333,120,100,5.524297,4.603581,0.454545
763,Patrick Maroon,1201.300000,106,92,5.294265,4.595022,0.464646
654,Michael Bunting,1277.650000,64,94,3.005518,4.414354,0.594937
783,Pierre-Luc Dubois,1466.516667,82,100,3.354889,4.091328,0.549451
331,Garnet Hathaway,1076.316667,65,73,3.623469,4.069434,0.528986
908,Tim StÃ¼tzle,1398.550000,37,91,1.587358,3.904043,0.710938
595,Marcus Foligno,1217.766667,89,78,4.385077,3.843101,0.467066
118,Brady Tkachuk,1399.566667,93,87,3.986948,3.729726,0.483333
915,Tom Wilson,1403.300000,78,86,3.334996,3.677047,0.524390
537,Kirill Kaprizov,1566.000000,36,94,1.379310,3.601533,0.723077
